In [89]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/kevingmagana/DSI/capstone/capstone-update/Capstone_Proposal/py_scripts/')
import results
from bs4 import BeautifulSoup
%reload_ext autoreload
%autoreload 2
import requests
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer




In [126]:
model[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4470 entries, 0 to 4469
Data columns (total 13 columns):
Unnamed: 0      4470 non-null int64
Unnamed: 0.1    4470 non-null int64
case_text       4470 non-null object
case_title      4470 non-null object
court           4470 non-null object
date            4470 non-null object
docket          4470 non-null object
tags            4470 non-null object
type_of_law     4470 non-null object
url             4470 non-null object
web_source      4470 non-null object
html_format     4470 non-null object
title_date      4470 non-null object
dtypes: int64(2), object(11)
memory usage: 648.9+ KB


In [7]:
df = pd.read_csv('/Users/kevingmagana/DSI/capstone/case_corpus.csv')
df = df.dropna()
df['title_date'] = df[['case_title', 'date']].apply(lambda x: ' '.join(x), axis=1)


df2 = df.iloc[:, [2, 12]]
df2 = df2.drop_duplicates(subset='title_date')
df2 = df2.reset_index(drop=True)


In [15]:


model = results.load_model()
def process_query(query):
    """
    :param query: string, the query
    :return: list, of indices or DOC IDs
      EX:
      	 [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]
    """
    # Step 1: Intake and get Indexed Cases
    query =  query
    indices = results.indexed_results(model[0], query)
    # Step 2: Get index_vectors and case_val dictionary
    index_vectors, case_value_dictionary= results.get_index_vectors(model[2], indices)
    # Step 3: Vectorize query
    vectorized_query = results.query_vector(model[1], query)
    # Step 4: Get Cosine similarities
    cosine_similarities = results.cosine_similarity(vectorized_query, index_vectors)
    # Step 5: Get top results and rank them (top 10 right now...)
    top_results = (results.get_top_values(cosine_similarities[0], 10), query)
    res = [case_value_dictionary[array_value] for array_value in top_results[0]]
    return res


In [32]:
model[3].case_text[0]

"United States Court of Appeals,Tenth Circuit.\nAlfredo Yero PORRO, Plaintiff-Appellant, v. Stanley BARNES; and Michael Bryant, in his official capacity as Sheriff of Jefferson County, Defendants-Appellees.\nNo. 10-6002.\nDecided: November 09, 2010\n\nBefore KELLY and GORSUCH, Circuit Judges, and MELGREN, District Judge.**Submitted on the briefs: *Mark E. Bialick and David B. Donchin, Durbin, Larimore & Bialick, Oklahoma City, OK; and David W. Little, Law Offices of David Little, Oklahoma City, OK, for Plaintiff-Appellant. Robert S. Lafferrandre and Randall J. Wood, Pierce Couch Hendrickson Baysinger & Green, L.L.P., Oklahoma City, OK; and Chris J. Collins, Timothy M. Melton, and Linda K. Soper, Collins, Zorn & Wagner, P.C., Oklahoma City, OK, for Defendants-Appellees.\nThis case presents two central questions: What provision of the Constitution should this court use to analyze a federal immigration detainee's claim of excessive force? And does a county's failure to adopt a prophylacti

In [226]:
html = model[3].html_format[4213]

# content = requests.get(html).content
soup = BeautifulSoup(html, 'lxml')


list_of_paragraphs = []
for paragaph in soup.findAll('p'): 
    list_of_paragraphs.append(str(paragaph.text.encode('ascii', 'ignore')).replace("'", ""))
# print(model[3].html_format[0])

len(list_of_paragraphs)

76

In [231]:
### Step 1: Vectorize and apply Tf-IDF weights on the case

cleaned_text = clean_text(list_of_paragraphs)

# tokenized= tokenize(list_of_paragraphs)
# remove_stops = remove_stop_words(tokenized)
# cleaned_text = stem_words(remove_stops)

list_of_strings = [" ".join(lst) for lst in cleaned_text]

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(list_of_strings)

vectors.shape, len(list_of_paragraphs)

((76, 1122), 76)

In [232]:
query = ['What is the penalty for drug trafficking?']
### Step 2: Vectorize the query

cleaned_query = clean_text(query)
query_list_of_strings = [" ".join(lst) for lst in cleaned_query]
tokenized_query = vectorizer.transform(query_list_of_strings)

cosine_similarities = linear_kernel(tokenized_query, vectors) 
top_results = (get_top_values(cosine_similarities[0], 10), query)
print(top_results)

([69, 20, 44, 40, 39, 14, 37, 46, 51, 68], ['What is the penalty for drug trafficking?'])


In [233]:
top_results[0]

[69, 20, 44, 40, 39, 14, 37, 46, 51, 68]

In [234]:
# print(tokenized_query.todense())
# vectorizer.vocabulary_
# print(list_of_strings)
# data= model[3].case_text[2939].split('\n')
print(query)
# print(list_of_strings)
print(list_of_paragraphs[69])


['What is the penalty for drug trafficking?']
b5.As noted above, aggravated felony includes the illicit trafficking in a controlled substance  including a drug trafficking crime (as defined in section 924(c) of Title 18).  Although it does not matter in application, we note that a drug offense with a trafficking element is illicit trafficking and a drug offense that is punishable as a felony under federal law is a drug trafficking crime.


In [435]:
def highlight_key_query_words(question, most_relevant_part):
    """
    :param question:
    :param most_relevant_part:
    :return:
    """
    # print(question)
    # print(most_relevant_part)
    final_results = []
    stop = set(stopwords.words('english'))
    parse_query = "".join(question).split(' ')
    parse_query = clean_text(parse_query)
    
    most_relevant = most_relevant_part.split(' ')
    check_result = clean_text(most_relevant)
    # print(len(most_relevant))
    # print(len(check_result))
    #
    # print(most_relevant)
    # print(check_result)

    for ind, word in enumerate(most_relevant):
        if check_result[ind] in parse_query and check_result[ind] != []:
#             final_results.append('<b>' + word + '</b>') ## If rendering on HTML
            final_results.append('\033[1m' + word + '\033[0m') ## Python
        else:
            final_results.append(word)

#     return " ".join(('...', ' '.join(final_results), '<b>' + '... Continue reading case' + '</b>' )) HTML
    return " ".join(('...', ' '.join(final_results), '\033[1m' + '... Continue reading case' + '\033[0m')) ## Python

def get_most_relevant_part(case_index, corpus_df, query, top_n=3, highlight_num=1):
    """
    :param case_index: int, the index of the case
    :param corpus_df: pandas dataframe, for the entire corpus of cases
    :param query: string, of the web-input query
    :param top_n: int, the top n matching paragraphs with query
    :returns query: the original query,
            top_result: list, of top indexed matching paragraphs,
            list_of_paragraphs:

    Two use cases: 
        (1) if highlight_num == 1 then it's to return for single highlighting. 
            See highlight_key_query_words() for example. 
        (2) if highlight_num > 1 then this function is for returning the list of 3 most
        relevant parts of a text to be highlighted once a user clicks on the search result. 
    
    """
    ## Step 1: Load up the data frame
    html = corpus_df.html_format[case_index]
    soup = BeautifulSoup(html, 'lxml')

    ## Step 2: Parse the case into a list of strings
    list_of_paragraphs = []
    for paragaph in soup.findAll('p'):
        list_of_paragraphs.append(paragaph.getText())

    ## Step 3: Parse case and vectorize with Tf-IDF weights
    cleaned_text = clean_text(list_of_paragraphs)
    list_of_strings = [" ".join(lst) for lst in cleaned_text]
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(list_of_strings)

    ## Step 4: Vectorize Query
#     query_list = [query]
    cleaned_query = clean_text(query)
    query_list_of_strings = [" ".join(lst) for lst in cleaned_query]
    tokenized_query = vectorizer.transform(query_list_of_strings)

    ## Step 5: Run Cosine Similarity on Query and Case Text
    cosine_similarities = linear_kernel(tokenized_query, vectors)
    top_results = [i for i in np.argsort(cosine_similarities[0])[-1:-top_n - 1:-1]]

    if highlight_num == 1: 
        highlight_output = list_of_paragraphs[top_results[0]]
    
    elif highlight_num == 2: 
        highlight_output = [list_of_paragraphs[ind] for ind in top_results[0:2]]  
    else: 
        highlight_output = [list_of_paragraphs[ind] for ind in top_results[0:3]]   
        
    return highlight_output  ## Returns top 3 indexed cases


## QUERY: What is the penalty for drug trafficking?
## Index of RESULTS [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]


In [286]:
# What is the penalty for drug trafficking?
## FLASK RESULT: [2269, 1219, 1416, 2458, 4468, 1769, 1948, 2035, 4466, 1927]
## result.py RESULT: [1537, 1659, 2446, 1761, 1868, 1927, 82, 1272, 864, 3177]




In [421]:

## Make sure query is in a LIST, otherwise WON'T WORK!
legal_query = ['What is the penalty for drug trafficking?']
data = model[3]
index = 2269
html = data.html_format[index]

results = get_most_relevant_part(index, data, legal_query, top_n=3, highlight_num = 1)
# print(results)
highlights = highlight_key_query_words(legal_query, results)
print(highlights)

# print(list_of_paragraphs[23])


... Trafficking in eight (8) or more ounces but less than five (5) pounds of marijuana is: ... Continue reading case


## Finding top 3 most relevant parts of the text

In [439]:

results = get_most_relevant_part(index, data, legal_query, top_n=3, highlight_num = 3)

print(len(results))

print(results)
for i in results: 
    print(i)


3
['Trafficking in eight (8) or more ounces but less than five (5) pounds of marijuana is:', "The BIA correctly determined that Garcia-Echaverria's Kentucky drug conviction constitutes an “aggravated felony” within the meaning of the INA. The term “aggravated felony” is defined through a list of qualifying offenses, which includes: \u2009“illicit trafficking in a controlled substance (as defined in section 802 of Title 21), including a drug trafficking crime (as defined in section 924(c) of Title 18).” \u20028 U.S.C. §\u20021101(a)(43)(B). \u2002In §\u2002924(c), “the term ‘drug trafficking crime’ means any felony punishable under the Controlled Substances Act (21 U.S.C. 801 et seq.),” or one of two other federal acts not relevant here. \u200218 U.S.C. §\u2002924(c)(2).", "Under the BIA's approach, a state drug conviction constitutes an “aggravated felony” under either of two routes. \u2002 Under the first route, a felony state drug conviction is an “aggravated felony” under §\u2002924

In [406]:
html = data.html_format[index]

# content = requests.get(html).content
soup = BeautifulSoup(html, 'lxml')

list_of_paragraphs = []
for paragaph in soup.findAll('p'): 
    list_of_paragraphs.append(paragaph.getText())
# print(model[3].html_format[0])

print(len(list_of_paragraphs))
print(list_of_paragraphs[23])



54
Trafficking in eight (8) or more ounces but less than five (5) pounds of marijuana is:


In [381]:
print(data.html_format[index])

[<div class="caselawcontent searchable-content">
<center>
<h2>United States Court of Appeals,Sixth Circuit.</h2>
<h3><p>Marco GARCIA-ECHAVERRIA, Petitioner-Appellant, v. UNITED STATES of America, Respondent-Appellee.</p></h3>
<h3>No. 03-3285.</h3>
<h3>    Decided: July 01, 2004</h3>
</center>
		Before:  MERRITT and MOORE, Circuit Judges;  DUGGAN, District Judge.  *Lawrence J. Kiroff (briefed), Assistant United States Attorney, Toledo, Ohio, for Appellee. Marco A. Garcia-Echaverria, El Paso, Texas, pro se.<!-- main body -->
<p>OPINION</p><p>Marco Garcia-Echaverria (“Garcia-Echaverria”), pro se Petitioner-Appellant, appeals the district court's denial of his petition for habeas corpus relief.   Garcia-Echaverria was sentenced for a conviction of unlawful reentry, in violation of 8 U.S.C. § 1326(b), and the Immigration and Naturalization Service (“INS,” now the Department of Homeland Security “DHS,”) has reinstated the prior Final Order of Removal.   On appeal, Garcia-Echaverria argues th

In [228]:
## QUERY: What is the penalty for drug trafficking?
## RESULTS [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]

def get_top_values(lst, n):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    
    [-1:-n-1:-1]
    '''
    return [i for i in np.argsort(lst)[-1:-n-1:-1]]

def most_relevant_part_of_one_text(case, query): 
    
    
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(case).toarray()
        
    tokenized_query = vectorizer.transform(query)
    cosine_similarities = results.cosine_similarity(vectorized_query, index_vectors)
    
    top_results = (results.get_top_values(cosine_similarities[0], 10), query)

    return top_results


def tokenize(tokenize_texts):
    """
    :param case_text: list of strings (case_texts)
    :returns: list of strings, tokenized word.
    """
    table = str.maketrans({key: None for key in string.punctuation})
    return [word_tokenize(case.translate(table).lower()) for case in tokenize_texts]

def remove_stop_words(remove_stop_text):
    """
    :param case_text: list of lists of strings (tokens)
    :returns: list of lists of strings(tokens) without stopwords
    """
    stop = set(stopwords.words('english'))
    return [[word for word in words if word not in stop] for words in remove_stop_text]

def stem_words(stem_word_text):
    """
    :param without_stop: list of lists without stop words
    :return: list of lists of strings (stemmed words)
    """
    snowball = SnowballStemmer('english')
    return [[snowball.stem(word) for word in words] for words in stem_word_text]


def clean_text(case_text):
    """
    :param case_text: list of strings (case_text)
    :return: list of lists of strings (stemmed words)
    """
    
    table = str.maketrans({key: None for key in string.punctuation})
    tokenized = [word_tokenize(case.translate(table).lower()) for case in case_text]
    stop = set(stopwords.words('english'))
    remove_stop_words = [[word for word in words if word not in stop] for words in tokenized]
    snowball = SnowballStemmer('english')
    ## Stemmed words 
    return [[snowball.stem(word) for word in words] for words in remove_stop_words]

# most_relevant_part_of_one_text(model[3].case_text[0], query)

# ## Step 

# vectorizer = TfidfVectorizer(stop_words='english')
# vectors = vectorizer.fit_transform(data).toarray()
